In [1]:
!pip install pandas matplotlib seaborn scipy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
cleanDatatest = pd.read_csv("../cleaned_dataset.csv")

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_machine_product_efficiency(df):
    """
    Analyze efficiency by machine and product to find optimal combinations
    """
    
    # Group by machine and product, calculate average efficiency
    efficiency_analysis = df.groupby(['maquina_id', 'producto_id']).agg({
        'eficiencia_porcentual': ['mean', 'std', 'count', 'min', 'max']
    }).round(2)
    
    # Flatten column names
    efficiency_analysis.columns = ['avg_efficiency', 'std_efficiency', 'count', 'min_efficiency', 'max_efficiency']
    efficiency_analysis = efficiency_analysis.reset_index()
    
    print("=== MACHINE-PRODUCT EFFICIENCY ANALYSIS ===")
    print("-" * 60)
    print(efficiency_analysis.to_string(index=False))
    
    return efficiency_analysis

def find_best_product_per_machine(df):
    """
    Find the best product for each machine based on average efficiency
    """
    
    # Calculate average efficiency by machine-product combination
    avg_efficiency = df.groupby(['maquina_id', 'producto_id'])['eficiencia_porcentual'].mean().reset_index()
    
    # Find the best product for each machine
    best_products = avg_efficiency.loc[avg_efficiency.groupby('maquina_id')['eficiencia_porcentual'].idxmax()]
    
    print("\n=== BEST PRODUCT FOR EACH MACHINE ===")
    print("-" * 45)
    
    for _, row in best_products.iterrows():
        machine = row['maquina_id']
        product = row['producto_id']
        efficiency = row['eficiencia_porcentual']
        
        # Get additional stats for this combination
        stats = df[(df['maquina_id'] == machine) & (df['producto_id'] == product)]['eficiencia_porcentual']
        
        print(f"Machine {machine}:")
        print(f"  Best Product: {product}")
        print(f"  Average Efficiency: {efficiency:.2f}%")
        print(f"  Data Points: {len(stats)}")
        print(f"  Efficiency Range: {stats.min():.2f}% - {stats.max():.2f}%")
        print()
    
    return best_products

def compare_all_combinations(df):
    """
    Create a detailed comparison table of all machine-product combinations
    """
    
    # Create pivot table for easy comparison
    pivot_table = df.pivot_table(
        values='eficiencia_porcentual', 
        index='maquina_id', 
        columns='producto_id', 
        aggfunc='mean'
    ).round(2)
    
    print("\n=== EFFICIENCY COMPARISON TABLE (Average %) ===")
    print("Rows: Machines, Columns: Products")
    print("-" * 50)
    print(pivot_table.to_string())
    
    # Find best and worst combinations
    melted = pivot_table.reset_index().melt(id_vars='maquina_id', var_name='producto_id', value_name='efficiency')
    melted = melted.dropna()
    
    best_combo = melted.loc[melted['efficiency'].idxmax()]
    worst_combo = melted.loc[melted['efficiency'].idxmin()]
    
    print(f"\n🏆 BEST COMBINATION:")
    print(f"   Machine {best_combo['maquina_id']} + Product {best_combo['producto_id']}: {best_combo['efficiency']:.2f}%")
    
    print(f"\n⚠️  WORST COMBINATION:")
    print(f"   Machine {worst_combo['maquina_id']} + Product {worst_combo['producto_id']}: {worst_combo['efficiency']:.2f}%")
    
    return pivot_table

def plot_efficiency_heatmap(df):
    """
    Create a heatmap showing efficiency by machine and product
    """
    pivot_data = df.pivot_table(
        values='eficiencia_porcentual', 
        index='maquina_id', 
        columns='producto_id', 
        aggfunc='mean'
    )
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_data, 
                annot=True, 
                fmt='.1f', 
                cmap='RdYlGn', 
                center=pivot_data.mean().mean(),
                cbar_kws={'label': 'Average Efficiency (%)'})
    
    plt.title('Machine-Product Efficiency Heatmap')
    plt.xlabel('Product ID')
    plt.ylabel('Machine ID')
    plt.tight_layout()
    plt.show()

def get_machine_rankings(df):
    """
    Rank products for each machine from best to worst efficiency
    """
    print("\n=== PRODUCT RANKINGS BY MACHINE ===")
    print("-" * 40)
    
    rankings = {}
    
    for machine in df['maquina_id'].unique():
        machine_data = df[df['maquina_id'] == machine]
        product_avg = machine_data.groupby('producto_id')['eficiencia_porcentual'].mean().sort_values(ascending=False)
        
        print(f"\nMachine {machine} - Product Rankings:")
        rankings[machine] = []
        
        for rank, (product, efficiency) in enumerate(product_avg.items(), 1):
            count = len(machine_data[machine_data['producto_id'] == product])
            print(f"  {rank}. Product {product}: {efficiency:.2f}% (n={count})")
            rankings[machine].append({
                'rank': rank,
                'product': product,
                'efficiency': efficiency,
                'count': count
            })
    
    return rankings


In [6]:
best_products = find_best_product_per_machine(cleanDatatest)


=== BEST PRODUCT FOR EACH MACHINE ===
---------------------------------------------
Machine M_1:
  Best Product: P_11
  Average Efficiency: 86.51%
  Data Points: 35
  Efficiency Range: 71.24% - 93.84%

Machine M_10:
  Best Product: P_8
  Average Efficiency: 87.21%
  Data Points: 31
  Efficiency Range: 75.78% - 94.39%

Machine M_2:
  Best Product: P_3
  Average Efficiency: 86.18%
  Data Points: 30
  Efficiency Range: 76.95% - 96.61%

Machine M_3:
  Best Product: P_13
  Average Efficiency: 87.14%
  Data Points: 27
  Efficiency Range: 78.97% - 95.46%

Machine M_4:
  Best Product: P_1
  Average Efficiency: 86.74%
  Data Points: 35
  Efficiency Range: 72.67% - 94.05%

Machine M_5:
  Best Product: P_11
  Average Efficiency: 86.38%
  Data Points: 33
  Efficiency Range: 75.38% - 95.51%

Machine M_6:
  Best Product: P_15
  Average Efficiency: 86.98%
  Data Points: 43
  Efficiency Range: 75.46% - 95.97%

Machine M_7:
  Best Product: P_3
  Average Efficiency: 86.92%
  Data Points: 26
  Efficienc

In [12]:
comparison_table = compare_all_combinations(cleanDatatest)


=== EFFICIENCY COMPARISON TABLE (Average %) ===
Rows: Machines, Columns: Products
--------------------------------------------------
producto_id    P_1   P_10   P_11   P_12   P_13   P_14   P_15    P_2    P_3    P_4    P_5    P_6    P_7    P_8    P_9
maquina_id                                                                                                          
M_1          84.35  84.35  86.51  84.90  84.86  84.07  84.96  82.94  85.31  85.02  84.80  85.03  83.72  84.07  85.23
M_10         85.50  83.66  83.06  86.81  83.65  85.10  84.57  84.91  84.58  84.44  84.56  84.11  84.58  87.21  86.59
M_2          84.30  84.49  86.04  86.10  84.88  86.07  83.92  83.75  86.18  85.19  86.16  85.02  83.41  84.54  83.32
M_3          85.17  85.14  84.65  84.22  87.14  86.01  83.07  85.51  85.32  84.70  83.80  84.44  84.58  84.03  83.57
M_4          86.74  84.80  84.65  83.62  84.76  85.84  84.46  84.82  86.11  83.57  85.81  84.74  85.42  84.83  85.48
M_5          84.84  84.04  86.38  84.90  84.35 

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_machine_operator_efficiency(df):
    """
    Analyze efficiency by machine and operator to find optimal combinations
    """
    
    # Group by machine and operator, calculate average efficiency
    efficiency_analysis = df.groupby(['maquina_id', 'operador_id']).agg({
        'eficiencia_porcentual': ['mean', 'std', 'count', 'min', 'max']
    }).round(2)
    
    # Flatten column names
    efficiency_analysis.columns = ['avg_efficiency', 'std_efficiency', 'count', 'min_efficiency', 'max_efficiency']
    efficiency_analysis = efficiency_analysis.reset_index()
    
    print("=== MACHINE-OPERATOR EFFICIENCY ANALYSIS ===")
    print("-" * 60)
    print(efficiency_analysis.to_string(index=False))
    
    return efficiency_analysis

def find_best_operator_per_machine(df):
    """
    Find the best operator for each machine based on average efficiency
    """
    
    # Calculate average efficiency by machine-operator combination
    avg_efficiency = df.groupby(['maquina_id', 'operador_id'])['eficiencia_porcentual'].mean().reset_index()
    
    # Find the best operator for each machine
    best_operators = avg_efficiency.loc[avg_efficiency.groupby('maquina_id')['eficiencia_porcentual'].idxmax()]
    
    print("\n=== BEST OPERATOR FOR EACH MACHINE ===")
    print("-" * 45)
    
    for _, row in best_operators.iterrows():
        machine = row['maquina_id']
        operator = row['operador_id']
        efficiency = row['eficiencia_porcentual']
        
        # Get additional stats for this combination
        stats = df[(df['maquina_id'] == machine) & (df['operador_id'] == operator)]['eficiencia_porcentual']
        
        print(f"Machine {machine}:")
        print(f"  Best Operator: {operator}")
        print(f"  Average Efficiency: {efficiency:.2f}%")
        print(f"  Data Points: {len(stats)}")
        print(f"  Efficiency Range: {stats.min():.2f}% - {stats.max():.2f}%")
        print()
    
    return best_operators

def compare_all_operator_combinations(df):
    """
    Create a detailed comparison table of all machine-operator combinations
    """
    
    # Create pivot table for easy comparison
    pivot_table = df.pivot_table(
        values='eficiencia_porcentual', 
        index='maquina_id', 
        columns='operador_id', 
        aggfunc='mean'
    ).round(2)
    
    print("\n=== EFFICIENCY COMPARISON TABLE (Average %) ===")
    print("Rows: Machines, Columns: Operators")
    print("-" * 50)
    print(pivot_table.to_string())
    
    # Find best and worst combinations
    melted = pivot_table.reset_index().melt(id_vars='maquina_id', var_name='operador_id', value_name='efficiency')
    melted = melted.dropna()
    
    best_combo = melted.loc[melted['efficiency'].idxmax()]
    worst_combo = melted.loc[melted['efficiency'].idxmin()]
    
    print(f"\n🏆 BEST COMBINATION:")
    print(f"   Machine {best_combo['maquina_id']} + Operator {best_combo['operador_id']}: {best_combo['efficiency']:.2f}%")
    
    print(f"\n⚠️  WORST COMBINATION:")
    print(f"   Machine {worst_combo['maquina_id']} + Operator {worst_combo['operador_id']}: {worst_combo['efficiency']:.2f}%")
    
    return pivot_table

def plot_operator_efficiency_heatmap(df):
    """
    Create a heatmap showing efficiency by machine and operator
    """
    pivot_data = df.pivot_table(
        values='eficiencia_porcentual', 
        index='maquina_id', 
        columns='operador_id', 
        aggfunc='mean'
    )
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_data, 
                annot=True, 
                fmt='.1f', 
                cmap='RdYlGn', 
                center=pivot_data.mean().mean(),
                cbar_kws={'label': 'Average Efficiency (%)'})
    
    plt.title('Machine-Operator Efficiency Heatmap')
    plt.xlabel('Operator ID')
    plt.ylabel('Machine ID')
    plt.tight_layout()
    plt.show()


def analyze_operator_performance_across_machines(df):
    """
    Show how each operator performs across different machines
    """
    print("\n=== OPERATOR PERFORMANCE ACROSS MACHINES ===")
    print("-" * 50)
    
    for operator in df['operador_id'].unique():
        operator_data = df[df['operador_id'] == operator]
        machine_avg = operator_data.groupby('maquina_id')['eficiencia_porcentual'].mean().sort_values(ascending=False)
        
        print(f"\nOperator {operator}:")
        print(f"  Overall Average: {operator_data['eficiencia_porcentual'].mean():.2f}%")
        print(f"  Best Machine: {machine_avg.idxmax()} ({machine_avg.max():.2f}%)")
        print(f"  Worst Machine: {machine_avg.idxmin()} ({machine_avg.min():.2f}%)")
        
        for machine, efficiency in machine_avg.items():
            count = len(operator_data[operator_data['maquina_id'] == machine])
            print(f"    Machine {machine}: {efficiency:.2f}% (n={count})")

def find_versatile_operators(df, threshold=80.0):
    """
    Find operators who perform well across multiple machines
    """
    print(f"\n=== VERSATILE OPERATORS (>{threshold}% on multiple machines) ===")
    print("-" * 60)
    
    versatile_operators = []
    
    for operator in df['operador_id'].unique():
        operator_data = df[df['operador_id'] == operator]
        machine_performance = operator_data.groupby('maquina_id')['eficiencia_porcentual'].mean()
        
        high_performance_machines = (machine_performance >= threshold).sum()
        avg_performance = machine_performance.mean()
        
        if high_performance_machines >= 2:  # Performs well on at least 2 machines
            versatile_operators.append({
                'operator': operator,
                'machines_above_threshold': high_performance_machines,
                'total_machines': len(machine_performance),
                'average_efficiency': avg_performance
            })
    
    # Sort by average efficiency
    versatile_operators.sort(key=lambda x: x['average_efficiency'], reverse=True)
    
    for op in versatile_operators:
        print(f"Operator {op['operator']}:")
        print(f"  High performance on {op['machines_above_threshold']}/{op['total_machines']} machines")
        print(f"  Overall average: {op['average_efficiency']:.2f}%")
        print()

    return versatile_operators


In [17]:
best_operators = find_best_operator_per_machine(cleanDatatest)


=== BEST OPERATOR FOR EACH MACHINE ===
---------------------------------------------
Machine M_1:
  Best Operator: OP_6
  Average Efficiency: 86.94%
  Data Points: 27
  Efficiency Range: 80.83% - 95.76%

Machine M_10:
  Best Operator: OP_7
  Average Efficiency: 86.58%
  Data Points: 31
  Efficiency Range: 72.56% - 97.94%

Machine M_2:
  Best Operator: OP_14
  Average Efficiency: 87.77%
  Data Points: 19
  Efficiency Range: 82.58% - 93.98%

Machine M_3:
  Best Operator: OP_19
  Average Efficiency: 87.65%
  Data Points: 24
  Efficiency Range: 77.51% - 95.46%

Machine M_4:
  Best Operator: OP_17
  Average Efficiency: 86.58%
  Data Points: 22
  Efficiency Range: 79.97% - 93.10%

Machine M_5:
  Best Operator: OP_8
  Average Efficiency: 86.79%
  Data Points: 18
  Efficiency Range: 75.28% - 95.50%

Machine M_6:
  Best Operator: OP_18
  Average Efficiency: 87.04%
  Data Points: 19
  Efficiency Range: 80.62% - 96.16%

Machine M_7:
  Best Operator: OP_3
  Average Efficiency: 86.27%
  Data Point

In [18]:
comparison_table = compare_all_operator_combinations(cleanDatatest)


=== EFFICIENCY COMPARISON TABLE (Average %) ===
Rows: Machines, Columns: Operators
--------------------------------------------------
operador_id   OP_1  OP_10  OP_11  OP_12  OP_13  OP_14  OP_15  OP_16  OP_17  OP_18  OP_19   OP_2  OP_20   OP_3   OP_4   OP_5   OP_6   OP_7   OP_8   OP_9
maquina_id                                                                                                                                             
M_1          84.22  84.97  86.33  84.29  84.12  84.24  84.70  84.85  84.42  84.62  83.95  85.70  86.49  83.47  83.48  84.33  86.94  85.29  84.25  82.44
M_10         84.38  84.21  86.24  84.17  84.68  85.00  84.85  84.24  85.47  84.65  85.14  84.05  85.46  82.87  83.36  85.25  84.99  86.58  85.60  84.33
M_2          86.14  83.22  84.06  84.46  84.52  87.77  85.30  86.34  85.70  84.15  85.32  84.86  86.14  83.94  83.65  85.22  84.65  85.75  83.61  83.99
M_3          84.11  84.10  84.55  85.77  85.46  83.94  84.37  83.27  83.44  84.43  87.65  84.18  86.47  8

In [ ]:
# Get the results from your two existing functions
best_operators = find_best_operator_per_machine(cleanDatatest)
best_products = find_best_product_per_machine(cleanDatatest)

# Merge them on machine_id
optimal_combinations = best_operators[['maquina_id', 'operador_id']].merge(
    best_products[['maquina_id', 'producto_id']], 
    on='maquina_id'
)

# Save to CSV
optimal_combinations.to_csv('optimal_combinations.csv', index=False)

print("CSV created with columns: maquina_id, operador_id, producto_id")
print(optimal_combinations)


=== BEST OPERATOR FOR EACH MACHINE ===
---------------------------------------------
Machine M_1:
  Best Operator: OP_6
  Average Efficiency: 86.94%
  Data Points: 27
  Efficiency Range: 80.83% - 95.76%

Machine M_10:
  Best Operator: OP_7
  Average Efficiency: 86.58%
  Data Points: 31
  Efficiency Range: 72.56% - 97.94%

Machine M_2:
  Best Operator: OP_14
  Average Efficiency: 87.77%
  Data Points: 19
  Efficiency Range: 82.58% - 93.98%

Machine M_3:
  Best Operator: OP_19
  Average Efficiency: 87.65%
  Data Points: 24
  Efficiency Range: 77.51% - 95.46%

Machine M_4:
  Best Operator: OP_17
  Average Efficiency: 86.58%
  Data Points: 22
  Efficiency Range: 79.97% - 93.10%

Machine M_5:
  Best Operator: OP_8
  Average Efficiency: 86.79%
  Data Points: 18
  Efficiency Range: 75.28% - 95.50%

Machine M_6:
  Best Operator: OP_18
  Average Efficiency: 87.04%
  Data Points: 19
  Efficiency Range: 80.62% - 96.16%

Machine M_7:
  Best Operator: OP_3
  Average Efficiency: 86.27%
  Data Point

In [19]:
cleanDatatest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986 entries, 0 to 4985
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              4986 non-null   object 
 1   turno                  4986 non-null   object 
 2   operador_id            4986 non-null   object 
 3   maquina_id             4986 non-null   object 
 4   producto_id            4986 non-null   object 
 5   temperatura            4986 non-null   float64
 6   vibración              4986 non-null   float64
 7   humedad                4986 non-null   float64
 8   tiempo_ciclo           4986 non-null   float64
 9   fallo_detectado        4986 non-null   object 
 10  tipo_fallo             495 non-null    object 
 11  cantidad_producida     4986 non-null   int64  
 12  unidades_defectuosas   4986 non-null   int64  
 13  eficiencia_porcentual  4986 non-null   float64
 14  consumo_energia        4986 non-null   float64
 15  para